# INSTALAÇÃO DAS BIBLIOTECAS

In [ ]:
%pip install pyspark

# IMPORTAÇÃO DAS BIBLIOTECAS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
import xml.etree.ElementTree as ET

# CRIAÇÃO DA APLICAÇÃO SPARK

In [ ]:
spark = SparkSession.builder.getOrCreate()

# VARIÁVEIS DE APOIO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
arquivo = '/content/drive/My Drive/ARQ/DADOS_ALUNOS.xml'

# FUNÇÃO PARA LER ARQUIVO XML

In [ ]:
def ler_xml(arquivo):
    tree = ET.parse(arquivo)
    root = tree.getroot()
    return root

# CRIAÇÃO DE LISTA COM O CONTEÚDO DO XML

In [ ]:
root = ler_xml(arquivo)
dados_alunos = []

for aluno in root:
  nome_completo = aluno.find('nome_completo').text
  data_nascimento = aluno.find('data_nascimento').text
  bairro = aluno.find('bairro').text
  cidade = aluno.find('cidade').text
  estado = aluno.find('estado').text
  tipo_escola = aluno.find('tipo_escola').text
  nome_escola = aluno.find('nome_escola').text
  for notas in aluno:
    for disciplinas in notas:
      disciplina = str(disciplinas.attrib.get('nome'))
      for atividades in disciplinas:
        atividade = atividades.find('nome').text
        valor = atividades.find('valor').text
        nota = atividades.find('nota').text
        dados_alunos.append([nome_completo, data_nascimento, bairro, cidade, estado, tipo_escola, nome_escola, disciplina, atividade, valor, nota])

# LEITURA DA LISTA USANDO SPARK

In [ ]:
df = spark.createDataFrame(dados_alunos, ['nome_completo', 'data_nascimento', 'bairro', 'cidade', 'estado', 'tipo_escola', 'nome_escola', 'disciplina', 'atividade', 'valor', 'nota'])

# EXIBINDO UMA AMOSTRA DOS DADOS

In [ ]:
df.show(truncate=False)

+----------------------+---------------+-----------+--------------+------------+-----------+-----------------------+----------+-----------+-----+----+
|nome_completo         |data_nascimento|bairro     |cidade        |estado      |tipo_escola|nome_escola            |disciplina|atividade  |valor|nota|
+----------------------+---------------+-----------+--------------+------------+-----------+-----------------------+----------+-----------+-----+----+
|Luiz Miguel Nascimento|09/02/2009     |Mangabeiras|Belo Horizonte|Minas Gerais|Municipal  |Escola Estadual Almeida|Português |Atividade 1|56   |52  |
|Luiz Miguel Nascimento|09/02/2009     |Mangabeiras|Belo Horizonte|Minas Gerais|Municipal  |Escola Estadual Almeida|Português |Atividade 2|31   |14  |
|Luiz Miguel Nascimento|09/02/2009     |Mangabeiras|Belo Horizonte|Minas Gerais|Municipal  |Escola Estadual Almeida|Português |Atividade 3|6    |1   |
|Luiz Miguel Nascimento|09/02/2009     |Mangabeiras|Belo Horizonte|Minas Gerais|Municipal  |Es

# EXIBINDO OS METADADOS (SCHEMA) DO ARQUIVO

In [ ]:
df.printSchema()

root
 |-- nome_completo: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- tipo_escola: string (nullable = true)
 |-- nome_escola: string (nullable = true)
 |-- disciplina: string (nullable = true)
 |-- atividade: string (nullable = true)
 |-- valor: string (nullable = true)
 |-- nota: string (nullable = true)



# AJUSTANDO O SCHEMA DOS DADOS, SE NECESSÁRIO

In [ ]:
df = df.withColumn('nota', df['nota'].cast('double'))
df = df.withColumn('valor', df['valor'].cast('double'))
df = df.withColumn("data_nascimento", to_date("data_nascimento", "dd/MM/yyyy"))
df = df.withColumn('data_nascimento', df['data_nascimento'].cast('date'))

df.printSchema()

root
 |-- nome_completo: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- tipo_escola: string (nullable = true)
 |-- nome_escola: string (nullable = true)
 |-- disciplina: string (nullable = true)
 |-- atividade: string (nullable = true)
 |-- valor: double (nullable = true)
 |-- nota: double (nullable = true)

